In [3]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline



# Load the dataset
in_file = 'titanic_data.csv'
full_data = pd.read_csv(in_file)
display(full_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Store the 'Survived' feature in a new variable and remove it from the dataset
Y = full_data['Survived']

features_raw = full_data.drop('Survived', axis = 1)
X = features_raw.drop('Name', axis = 1)

# Show the new dataset with 'Survived' removed
display(X.head())

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,3,male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
features = pd.get_dummies(X)

In [6]:
features = features.fillna(0.0)
display(features.head())

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Ticket_110152,Ticket_110413,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.0,1,0,7.2500,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,3,26.0,0,0,7.9250,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,3,35.0,0,0,8.0500,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=42)

In [24]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid = {
    'max_depth':[2,6,10,14],
    'max_features':range(1,1001,100),
    'min_samples_leaf':[1,2,4,6,8,10],
    'min_samples_split':[2,4,6,8,10],
    'n_estimators':range(2,10,2)
        
}

cv_sets = ShuffleSplit(n_splits=3, test_size=0.2,train_size=None)

scorer  = make_scorer(f1_score) 

grid_search = GridSearchCV( estimator = rf, param_grid = param_grid, cv = cv_sets, n_jobs = -1, verbose = 1 )

grid_fit = grid_search.fit(X_train,y_train)

best_clf = grid_fit.best_estimator_

Fitting 3 folds for each of 4800 candidates, totalling 14400 fits


In [25]:
best_train_predictons = best_clf.predict(X_train)

best_test_predictons  = best_clf.predict(X_test)

print('training F1 Score =', f1_score(best_train_predictons,y_train))
print('testing F1 Score =', f1_score(best_test_predictons,y_test))

train_acc = accuracy_score(y_train,best_train_predictons)
test_acc = accuracy_score(y_test,best_test_predictons)

print('training accuracy =', train_acc)
print('testing accuracy =', test_acc)

best_clf

training F1 Score = 0.8343815513626834
testing F1 Score = 0.7384615384615384
training accuracy = 0.8890449438202247
testing accuracy = 0.8100558659217877


RandomForestClassifier(max_depth=10, max_features=201, min_samples_split=6,
                       n_estimators=4, random_state=42)